# Code Python pour le Projet d'Algèbre

## Chargement des bibliothèques

In [3]:
import csv
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from math import exp

## Définition de la classe DataMatrix

Cette classe va comprendre les attributs et méthodes nécéssaires aux diverses questions du Projet

### 1. Import des données et normalisation

L'import des données se fait par la lecture du fichier `.csv` fourni.<br/>
On normalise les données en effectuant 

$\cfrac{x_{ij} - j_{moy}}{j_{max}-j_{min}}$

Sur chaque $x_{ij}$, élément de nos données de nos données ; avec $i$ une ligne et $j$ une colonne.

### 2. Construction de la matrice $L$

Soient  $x, y$ deux éléments de notre tableau de données.<br/>
On renvoie le résultat de $k(x,y)$ pour tout élément de notre tableau, et construisons ainsi la matrice symétrique $L$
(car $k(x,y) = k(y,x)$).


### 3. Construction de la matrice $D$
$D$ est une matrice diagonnale telle que :

$D_{ii} = \sum_{j=1}L_{ij}$

et

$D_{ij} = 0 , i \neq j$

### 4. Construction de la matrice $M$

On construit M, matrice stochastique avec :

$M = D^{-1/2}LD^{1/2}$

### 5. Définition de la fonction noyau de similarité $k(x,y)$
Cette fonction renvoie le noyau de similarité gaussien au paramètre $\epsilon$ défini:

${\Large e^{-\frac{||x-y||^2}{\epsilon}}}$


In [ ]:
class DataMatrix:
  def __init__(self, eps, filename):
    
    self.eps = eps
    
#1. Import des données et normalisation   
#--------------
    self.data = pd.read_csv(filename, decimal=",", sep="\s+")

    #Stockage des en-têtes
    self.headers = list(self.data.columns.values)
    
    #Stockage des valeurs
    self.data = np.array(self.data.values)
    self.nb_el = len(self.data)

    
    for i, row  in enumerate(self.data[:,1:]) :
      for j, elt in enumerate(row):
        
        #Transformation des objets en float
        elt = float(elt)
        
        #Normalisation des données
        self.data[i,j+1] = (self.data[i,j+1] - self.data[:,j+1].mean())/(self.data[:,j+1].max()-self.data[:,j+1].min()) if self.data[:,j+1].max()-self.data[:,j+1].min() != 0 else 0 #(self.data[i,j] - self.data[:,j].mean())


#2. Construction de la matrice L
#--------------
    self.L = np.array([[0.0  for _ in range(self.nb_el)] for _ in range(self.nb_el)])

    for i in range(self.nb_el):
      for j in range(self.nb_el):
        if i >= j :
          self.L[i,j] = self.k(i,j)
          self.L[j,i] = self.L[i,j]

#3. Construction de la matrice D
#--------------
    self.D = np.array([[0.0  for _ in range(self.nb_el)] for _ in range(self.nb_el)])

    for i in range(self.nb_el):
      self.D[i,i] = self.L[i].sum()


#4. Construction de la matrice M avec M = D^-1/2 L D^1/2
#--------------
    D_half = np.matrix(np.zeros(self.D.shape))
    D_mhalf = np.matrix(np.zeros(self.D.shape))

    np.fill_diagonal(D_half, 1/ (self.D.diagonal()**0.5))    # -> D^1/2
    np.fill_diagonal(D_mhalf, 1/ (self.D.diagonal()**-0.5))  # -> D^-1/2


    self.M = np.dot(D_mhalf,self.L).dot(D_half)

      #Construction de M si M = D^-1 L
      #self.M = np.linalg.inv(self.D).dot(self.L)
    

#5. Définition de la fonction noyau de similarité
#--------------
    
  def k(self, i ,j):
    """Returns the gaussian similarity kernel between the elements at positions i and j"""
    x = self.data[i,1:]
    y = self.data[j,1:]
    return exp(-((np.linalg.norm(x-y))**2)/self.eps)

In [5]:
datatest = DataMatrix(2,"data_small.csv")
print datatest.L

[[  1.00000000e+00   6.13322942e-03   2.87927446e-03 ...,   2.91267288e-06
    2.75487208e-11   3.52387194e-09]
 [  6.13322942e-03   1.00000000e+00   4.56332250e-01 ...,   4.38678882e-04
    6.98146139e-07   6.01214598e-06]
 [  2.87927446e-03   4.56332250e-01   1.00000000e+00 ...,   4.62020517e-04
    4.37356025e-06   2.35642823e-05]
 ..., 
 [  2.91267288e-06   4.38678882e-04   4.62020517e-04 ...,   1.00000000e+00
    1.44817240e-05   5.63719919e-03]
 [  2.75487208e-11   6.98146139e-07   4.37356025e-06 ...,   1.44817240e-05
    1.00000000e+00   4.99876711e-04]
 [  3.52387194e-09   6.01214598e-06   2.35642823e-05 ...,   5.63719919e-03
    4.99876711e-04   1.00000000e+00]]
